In [1]:
import pandas as pd
import torch
from llama_recipes.inference.model_utils import load_model

from rtfm.configs import TrainConfig, TokenizerConfig
from rtfm.inference_utils import InferenceModel
from rtfm.serialization.serializers import get_serializer
from rtfm.tokenization.text import prepare_tokenizer
from transformers import AutoTokenizer, LlamaForCausalLM, AutoConfig

train_config = TrainConfig(model_name="mlfoundations/tabula-8b", context_length=8192)

# TODO(jpgard): set add_serializer_tokens to True when using TabuLa-8B
tokenizer_config = TokenizerConfig()

# Load the configuration
config = AutoConfig.from_pretrained(train_config.model_name)

# Set the torch_dtype to bfloat16
config.torch_dtype = 'bfloat16'

device = "cuda" if torch.cuda.is_available() else "cpu"


model = LlamaForCausalLM.from_pretrained(train_config.model_name, device_map="auto", config=config).to(device)

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
serializer = get_serializer(train_config.serializer_cls)
tokenizer, model = prepare_tokenizer(
    model,
    tokenizer=tokenizer,
    pretrained_model_name_or_path=train_config.model_name,
    model_max_length=train_config.context_length,
    use_fast_tokenizer=tokenizer_config.use_fast_tokenizer,
    serializer_tokens_embed_fn=tokenizer_config.serializer_tokens_embed_fn,
    serializer_tokens=serializer.special_tokens
    if tokenizer_config.add_serializer_tokens
    else None,
)

inference_model = InferenceModel(model=model, tokenizer=tokenizer, serializer=serializer)

read user yaml files: 0it [00:00, ?it/s]
/gscratch/efml/jpgard/miniconda3/envs/rtfm/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
labeled_examples = pd.DataFrame(
    [{"name": "Tadej Pogačar", 
      "occupation": "cyclist",
      "height_in_meters": 1.76, 
      "weight_in_kg": 66,
     "bike": "Colnago V4Rs",
     "fitness_level": "excellent",
     },
     {"name": "Josh Gardner", 
      "occupation": "student",
      "height_in_meters": 1.88, 
      "weight_in_kg": 72,
     "bike": "Cannondale Synapse",
     "fitness_level": "good",
     },]
)
target_example = pd.DataFrame(
    [
    {"name": "Ludwig Schmidt", 
     "occupation": "Assistant professor of computer science",
     "height_in_meters": 1.8, 
     "weight_in_kg": 68,
     # "bike": "Cervelo",
     "bike": "Colnago V4Rs",
     },
    ]
)

output = inference_model.predict(
    target_example=target_example,
    target_colname="fitness_level",
    target_choices=["poor", "fair", "good", "excellent"],
    labeled_examples=labeled_examples,
    handle_invalid_predictions="warn",
)
print(f"prediction is: {output}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


prediction is: excellent


In [ ]:
list(model.parameters())[0].device